In [2]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

class Solution:
    """The solution of Port Scheduling Problem with multi-tugboat service support.
    Contains vessel assignments, tugboat assignments, and timing information.
    
    Attributes:
        vessel_assignments (dict): Maps vessel_id to (berth_id, start_time) tuple. 
                                 None if vessel is unassigned.
        tugboat_inbound_assignments (dict): Maps vessel_id to list of (tugboat_id, start_time) tuples for inbound service.
        tugboat_outbound_assignments (dict): Maps vessel_id to list of (tugboat_id, start_time) tuples for outbound service.
    """
    def __init__(self, vessel_assignments: dict = None, 
                 tugboat_inbound_assignments: dict = None, 
                 tugboat_outbound_assignments: dict = None):
        self.vessel_assignments = vessel_assignments or {}
        self.tugboat_inbound_assignments = tugboat_inbound_assignments or {}
        self.tugboat_outbound_assignments = tugboat_outbound_assignments or {}

    def __str__(self) -> str:
        result = "Port Scheduling Solution:\n"
        result += "Vessel Assignments:\n"
        for vessel_id, assignment in self.vessel_assignments.items():
            if assignment is not None:
                berth_id, start_time = assignment
                result += f"  Vessel {vessel_id}: Berth {berth_id}, Start Time {start_time}\n"
            else:
                result += f"  Vessel {vessel_id}: Unassigned\n"
        
        result += "Inbound Tugboat Services:\n"
        for vessel_id, services in self.tugboat_inbound_assignments.items():
            if services:
                service_str = ", ".join([f"Tugboat {tug_id} at time {start_time}" 
                                       for tug_id, start_time in services])
                result += f"  Vessel {vessel_id}: {service_str}\n"
        
        result += "Outbound Tugboat Services:\n"
        for vessel_id, services in self.tugboat_outbound_assignments.items():
            if services:
                service_str = ", ".join([f"Tugboat {tug_id} at time {start_time}" 
                                       for tug_id, start_time in services])
                result += f"  Vessel {vessel_id}: {service_str}\n"
        
        return result

def load_case_data(filename):
    """Load case data from txt file."""
    data = {}
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#'):
                if '=' in line:
                    key, value = line.split('=', 1)
                    key = key.strip()
                    value = value.strip()
                    
                    # Parse different data types
                    if value.startswith('[') and value.endswith(']'):
                        # List data
                        value = value[1:-1]  # Remove brackets
                        if ',' in value:
                            data[key] = [float(x.strip()) if '.' in x else int(x.strip()) 
                                       for x in value.split(',')]
                        else:
                            data[key] = [float(value) if '.' in value else int(value)]
                    else:
                        # Single value
                        try:
                            data[key] = float(value) if '.' in value else int(value)
                        except ValueError:
                            data[key] = value
    return data

def solve_port_scheduling_horsepower_gurobi(data_file):
    """Solve port scheduling problem with horsepower constraints using Gurobi."""
    
    # Load data
    data = load_case_data(data_file)
    
    # Extract parameters
    I = data['vessel_num']  # Number of vessels
    J = data['berth_num']   # Number of berths  
    K = data['tugboat_num'] # Number of tugboats
    T = data['time_periods'] # Time periods
    
    # Vessel parameters
    S = data['vessel_sizes']  # S_i: vessel size levels
    ETA = data['vessel_etas']  # ETA_i
    D = data['vessel_durations']  # D_i
    tau_in = data['vessel_inbound_service_times']  # τ^in_i
    tau_out = data['vessel_outbound_service_times']  # τ^out_i
    P_req = data['vessel_horsepower_requirements']  # P^req_i: horsepower requirements
    Delta_early = data['vessel_early_limits']  # Δ^early_i
    Delta_late = data['vessel_late_limits']  # Δ^late_i
    alpha = data['vessel_priority_weights']  # α_i
    beta = data['vessel_waiting_costs']  # β_i
    gamma = data['vessel_jit_costs']  # γ_i
    
    # Berth parameters
    C = data['berth_capacities']  # C_j: berth capacity levels
    
    # Tugboat parameters
    P_k = data['tugboat_horsepower']  # P_k: tugboat horsepower
    c_k = data['tugboat_costs']  # c_k: tugboat costs
    
    # System parameters
    rho_in = data['inbound_preparation_time']  # ρ^in
    rho_out = data['outbound_preparation_time']  # ρ^out
    H_max = data['max_tugboats_per_service']  # H_max: max tugboats per service
    epsilon_time = data['time_constraint_tolerance']  # ε_time
    M = data['penalty_parameter']  # Big number
    lambda_1 = data['objective_weights'][0]
    lambda_2 = data['objective_weights'][1] 
    lambda_3 = data['objective_weights'][2]
    lambda_4 = data['objective_weights'][3]
    
    # Create model
    model = gp.Model("PortSchedulingHorsepower")
    
    # Decision variables
    # x[i,j,t]: vessel i starts berthing at berth j at time t
    x = model.addVars(I, J, T, vtype=GRB.BINARY, name="x")
    
    # y_in[i,k,t]: tugboat k starts inbound service for vessel i at time t
    y_in = model.addVars(I, K, T, vtype=GRB.BINARY, name="y_in")
    
    # y_out[i,k,t]: tugboat k starts outbound service for vessel i at time t  
    y_out = model.addVars(I, K, T, vtype=GRB.BINARY, name="y_out")
    
    # z_in[i,t]: auxiliary variable - vessel i starts inbound service at time t
    z_in = model.addVars(I, T, vtype=GRB.BINARY, name="z_in")
    
    # z_out[i,t]: auxiliary variable - vessel i starts outbound service at time t
    z_out = model.addVars(I, T, vtype=GRB.BINARY, name="z_out")
    
    # u_early[i], u_late[i]: ETA deviation variables
    u_early = model.addVars(I, vtype=GRB.CONTINUOUS, name="u_early")
    u_late = model.addVars(I, vtype=GRB.CONTINUOUS, name="u_late")
    
    # Objective function components
    # Z1: Unserved vessel penalty
    Z1 = gp.quicksum(M * alpha[i] * (1 - gp.quicksum(x[i,j,t] for j in range(J) for t in range(T))) 
                     for i in range(I))
    
    # Z2: Total port time cost (using auxiliary variables)
    Z2 = gp.quicksum(alpha[i] * beta[i] * (
        gp.quicksum((t + tau_out[i]) * z_out[i,t] for t in range(T)) - 
        gp.quicksum(t * z_in[i,t] for t in range(T))
    ) for i in range(I))
    
    # Z3: ETA deviation cost
    Z3 = gp.quicksum(alpha[i] * gamma[i] * (u_early[i] + u_late[i]) for i in range(I))
    
    # Z4: Tugboat utilization cost
    Z4 = gp.quicksum(c_k[k] * (tau_in[i] * y_in[i,k,t] + tau_out[i] * y_out[i,k,t]) 
                     for k in range(K) for i in range(I) for t in range(T))
    
    # Set objective
    model.setObjective(lambda_1 * Z1 + lambda_2 * Z2 + lambda_3 * Z3 + lambda_4 * Z4, GRB.MINIMIZE)
    
    # Constraints
    
    # Constraint (1): Each vessel can be assigned at most once
    for i in range(I):
        model.addConstr(gp.quicksum(x[i,j,t] for j in range(J) for t in range(T)) <= 1, 
                       name=f"vessel_assignment_{i}")
    
    # Constraint (2): Vessel-berth compatibility
    for i in range(I):
        for j in range(J):
            for t in range(T):
                if C[j] < S[i]:
                    model.addConstr(x[i,j,t] == 0, name=f"berth_compatibility_{i}_{j}_{t}")
    
    # Constraint (3): Inbound tugboat service coupling
    for i in range(I):
        model.addConstr(gp.quicksum(z_in[i,t] for t in range(T)) == 
                       gp.quicksum(x[i,j,t] for j in range(J) for t in range(T)), 
                       name=f"inbound_coupling_{i}")
    
    # Constraint (4): Outbound tugboat service coupling  
    for i in range(I):
        model.addConstr(gp.quicksum(z_out[i,t] for t in range(T)) == 
                       gp.quicksum(x[i,j,t] for j in range(J) for t in range(T)), 
                       name=f"outbound_coupling_{i}")
    
    # Constraint (5): Tugboat horsepower constraints for inbound
    for i in range(I):
        for t in range(T):
            model.addConstr(gp.quicksum(P_k[k] * y_in[i,k,t] for k in range(K)) >= 
                           P_req[i] * z_in[i,t], 
                           name=f"horsepower_inbound_{i}_{t}")
    
    # Constraint (6): Tugboat horsepower constraints for outbound
    for i in range(I):
        for t in range(T):
            model.addConstr(gp.quicksum(P_k[k] * y_out[i,k,t] for k in range(K)) >= 
                           P_req[i] * z_out[i,t], 
                           name=f"horsepower_outbound_{i}_{t}")
    
    # Constraint (7): Tugboat quantity limits for inbound
    for i in range(I):
        for t in range(T):
            model.addConstr(gp.quicksum(y_in[i,k,t] for k in range(K)) <= 
                           H_max * z_in[i,t], 
                           name=f"tugboat_limit_inbound_{i}_{t}")
    
    # Constraint (8): Tugboat quantity limits for outbound
    for i in range(I):
        for t in range(T):
            model.addConstr(gp.quicksum(y_out[i,k,t] for k in range(K)) <= 
                           H_max * z_out[i,t], 
                           name=f"tugboat_limit_outbound_{i}_{t}")
    
    # Constraint (9): Auxiliary variable definition for inbound
    for i in range(I):
        for t in range(T):
            model.addConstr(z_in[i,t] <= gp.quicksum(y_in[i,k,t] for k in range(K)), 
                           name=f"aux_inbound_{i}_{t}")
    
    # Constraint (10): Auxiliary variable definition for outbound
    for i in range(I):
        for t in range(T):
            model.addConstr(z_out[i,t] <= gp.quicksum(y_out[i,k,t] for k in range(K)), 
                           name=f"aux_outbound_{i}_{t}")
    
    # Constraint (11): Berth capacity constraints
    for j in range(J):
        for t in range(T):
            model.addConstr(gp.quicksum(x[i,j,tau] 
                           for i in range(I) 
                           for tau in range(max(0, t-D[i]+1), t+1)) <= 1, 
                           name=f"berth_capacity_{j}_{t}")
    
    # Constraint (12): Tugboat capacity constraints with preparation time
    for k in range(K):
        for t in range(T):
            model.addConstr(
                gp.quicksum(y_in[i,k,tau] 
                           for i in range(I) 
                           for tau in range(max(0, t-tau_in[i]-rho_in+1), t+1)) +
                gp.quicksum(y_out[i,k,tau] 
                           for i in range(I) 
                           for tau in range(max(0, t-tau_out[i]-rho_out+1), t+1)) <= 1,
                name=f"tugboat_capacity_{k}_{t}")
    
    # Constraint (13): Inbound service time window constraints
    for i in range(I):
        for k in range(K):
            for t in range(T):
                if t < ETA[i] - Delta_early[i] or t > ETA[i] + Delta_late[i]:
                    model.addConstr(y_in[i,k,t] == 0, name=f"time_window_in_{i}_{k}_{t}")
    
    # Constraint (14): Inbound-berthing timing sequence constraints
    for i in range(I):
        berth_start = gp.quicksum(t * x[i,j,t] for j in range(J) for t in range(T))
        inbound_end = gp.quicksum((t + tau_in[i]) * z_in[i,t] for t in range(T))
        vessel_assigned = gp.quicksum(x[i,j,t] for j in range(J) for t in range(T))
        
        model.addConstr(berth_start - inbound_end >= 0, 
                       name=f"inbound_berth_lower_{i}")
        model.addConstr(berth_start - inbound_end <= epsilon_time * vessel_assigned, 
                       name=f"inbound_berth_upper_{i}")
    
    # Constraint (15): Berthing-outbound timing sequence constraints
    for i in range(I):
        outbound_start = gp.quicksum(t * z_out[i,t] for t in range(T))
        berth_end = gp.quicksum((t + D[i]) * x[i,j,t] for j in range(J) for t in range(T))
        vessel_assigned = gp.quicksum(x[i,j,t] for j in range(J) for t in range(T))
        
        model.addConstr(outbound_start - berth_end >= 0, 
                       name=f"berth_outbound_lower_{i}")
        model.addConstr(outbound_start - berth_end <= epsilon_time * vessel_assigned, 
                       name=f"berth_outbound_upper_{i}")
    
    # Constraint (16): ETA deviation linearization constraints
    for i in range(I):
        inbound_start = gp.quicksum(t * z_in[i,t] for t in range(T))
        model.addConstr(inbound_start == ETA[i] + u_late[i] - u_early[i], 
                       name=f"eta_deviation_{i}")
    
    # Solve the model
    model.setParam('TimeLimit', 60*30)  # 30 minutes time limit
    model.setParam('MIPGap', 0.02)      # 2% optimality gap
    model.setParam('Threads', 4)        # Use 4 threads
    model.optimize()
    
    # Extract and return solution in Solution class format
    if model.Status == GRB.OPTIMAL or model.Status == GRB.TIME_LIMIT:
        print(f"Solution Status: {model.Status}")
        print(f"Objective Value: {model.ObjVal:.2f}")
        
        # Extract vessel assignments
        vessel_assignments = {}
        for i in range(I):
            vessel_assignments[i] = None  # Default to unassigned
            for j in range(J):
                for t in range(T):
                    if x[i,j,t].X > 0.5:
                        vessel_assignments[i] = (j, t)
                        break
                if vessel_assignments[i] is not None:
                    break
        
        # Extract tugboat inbound assignments (supporting multiple tugboats)
        tugboat_inbound_assignments = {}
        for i in range(I):
            services = []
            for k in range(K):
                for t in range(T):
                    if y_in[i,k,t].X > 0.5:
                        services.append((k, t))
            tugboat_inbound_assignments[i] = services
        
        # Extract tugboat outbound assignments (supporting multiple tugboats)
        tugboat_outbound_assignments = {}
        for i in range(I):
            services = []
            for k in range(K):
                for t in range(T):
                    if y_out[i,k,t].X > 0.5:
                        services.append((k, t))
            tugboat_outbound_assignments[i] = services
        
        # Create Solution object
        solution = Solution(
            vessel_assignments=vessel_assignments,
            tugboat_inbound_assignments=tugboat_inbound_assignments,
            tugboat_outbound_assignments=tugboat_outbound_assignments
        )
        
        # Print solution using Solution class format
        print("\n" + str(solution))
        
        # Print objective breakdown
        print("=== OBJECTIVE BREAKDOWN ===")
        z1_val = sum(M * alpha[i] * (1 - sum(x[i,j,t].X for j in range(J) for t in range(T))) 
                     for i in range(I))
        z2_val = sum(alpha[i] * beta[i] * (
            sum((t + tau_out[i]) * z_out[i,t].X for t in range(T)) - 
            sum(t * z_in[i,t].X for t in range(T))
        ) for i in range(I))
        z3_val = sum(alpha[i] * gamma[i] * (u_early[i].X + u_late[i].X) for i in range(I))
        z4_val = sum(c_k[k] * (tau_in[i] * y_in[i,k,t].X + tau_out[i] * y_out[i,k,t].X) 
                     for k in range(K) for i in range(I) for t in range(T))
        
        print(f"Z1 (Unserved penalty): {z1_val:.2f}")
        print(f"Z2 (Port time cost): {z2_val:.2f}")
        print(f"Z3 (ETA deviation cost): {z3_val:.2f}")
        print(f"Z4 (Tugboat cost): {z4_val:.2f}")
        print(f"Total Objective: {model.ObjVal:.2f}")
        
        # Print horsepower analysis
        print("\n=== HORSEPOWER ANALYSIS ===")
        for i in range(I):
            if vessel_assignments[i] is not None:
                inbound_hp = sum(P_k[k] for k, t in tugboat_inbound_assignments[i])
                outbound_hp = sum(P_k[k] for k, t in tugboat_outbound_assignments[i])
                required_hp = P_req[i]
                print(f"Vessel {i}: Required={required_hp}HP, Inbound={inbound_hp}HP, Outbound={outbound_hp}HP")
        
        return solution, model
    else:
        print(f"No solution found. Status: {model.Status}")
        return None, None

if __name__ == "__main__":
    # Example usage - update with actual file path
    solution, model = solve_port_scheduling_horsepower_gurobi(
        r"C:\Users\gongh\Desktop\变体3测试数据\100_15_30_T24.txt")

    if solution is not None:
        print(f"\nSolution successfully obtained!")
        print(f"Gurobi model status: {model.Status}")
        # You can now use the solution object for further analysis
    else:
        print("Failed to obtain solution.")

Set parameter TimeLimit to value 1800
Set parameter MIPGap to value 0.02
Set parameter Threads to value 4
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12450H, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 12 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  1800
MIPGap  0.02
Threads  4

Optimize a model with 87740 rows, 185000 columns and 1242275 nonzeros
Model fingerprint: 0xeb18215e
Variable types: 200 continuous, 184800 integer (184800 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [3e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 118239.12119
Presolve removed 80202 rows and 74488 columns
Presolve time: 1.46s
Presolved: 7538 rows, 110512 columns, 606100 nonzeros
Variable types: 0 continuous, 110512 integer (110412 binary)
Performing another presolve...
Pr